Libraries

In [115]:
import pandas as pd
import requests
from datetime import datetime
import time
import os
import json

Bring in data

In [79]:
capital_cities = pd.read_csv('../Data/capital_cities_and_airports.csv')

display(capital_cities.shape)
display(capital_cities.head())

(45, 6)

,country,capital_city,capital_latitude,capital_longitude,airport_iata,airport_name
0,Afghanistan,Kabul,34.5167,69.1833,KBL,Hamid Karzai International Airport
1,Armenia,Yerevan,40.1812,44.5136,EVN,Zvartnots International Airport
2,Azerbaijan,Baku,40.3953,49.8622,GYD,Heydar Aliyev International Airport
3,Bahrain,Manama,26.2361,50.5831,BAH,Bahrain International Airport
4,Bangladesh,Dhaka,23.7231,90.4086,DAC,Hazrat Shahjalal International Airport


Helper functions

In [3]:
# format departure datetime
def combine_to_datetime(dataframe, direction):
    # combine the departure columns into a single datetime column
    dataframe[f'{direction}_datetime'] = pd.to_datetime(
        dataframe[f'{direction}_year'].astype(str) + '-' +
        dataframe[f'{direction}_month'].astype(str).str.zfill(2) + '-' +
        dataframe[f'{direction}_day'].astype(str).str.zfill(2) + ' ' +
        dataframe[f'{direction}_hour'].astype(str).str.zfill(2) + ':' +
        dataframe[f'{direction}_minute'].astype(str).str.zfill(2) + ':' +
        dataframe[f'{direction}_second'].astype(str).str.zfill(2)
    )
    
    # drop the original departure columns
    dataframe.drop([f'{direction}_day', f'{direction}_month', f'{direction}_year', f'{direction}_hour', f'{direction}_minute', f'{direction}_second'], axis=1, inplace=True)
    

Get data

In [4]:
# bring in key
with open('../Data/skyscanner_auth.txt', 'r') as file:
    skyscanner_auth = file.read()


In [119]:
# set up api
def get_flight_tables(departure_airport, arrival_airport, year, month, day, skyscanner_auth=skyscanner_auth):
    url = 'https://partners.api.skyscanner.net/apiservices/v3/flights/live/search/create'
    headers = {'x-api-key': skyscanner_auth}

    data = {
        'query': {
            'market': 'US',
            'locale': 'en-US',
            'currency': 'USD',
            'query_legs': [{
                'origin_place_id': {'iata': departure_airport},
                'destination_place_id': {'iata': arrival_airport},
                'date': {'year': year, 'month': month, 'day': day}
            }],
            'adults': 1,
            'cabin_class': 'CABIN_CLASS_ECONOMY'
        }
    }
    
    try:
        response = requests.post(url, headers=headers, json=data)
        
        if response.status_code == 200:
            # store data
            response = response.json()
            #print(json.dumps(response, indent=4)

            return response['content']['results']
        else:
            response.raise_for_status()
    except requests.exceptions.HTTPError as err:
        print(err)

# test
flight_tables = get_flight_tables('NRT', 'TLV', 2023, 5, 22)


* Itineraries:	Bookable itinerary which corresponds with what was requested in the search. A return trip will consist of 2 legs, while a one-way trip will consist of 1 leg. An itinerary will contain a deepLink field which takes the traveler to the booking page.
* Leg:	Includes details about the flight leg from destination to origin. A leg has 1 segment if it is a direct flight, and can have multiple segments if there are multiple stopovers.
* Segment:	Shows the individual stops in a leg. I.e.: if a leg has 1 stop, the segment will show details about the stopover such as the length of time and where the stopover location is.
* Places:	Shows the individual stops in a leg. I.e.: if a leg has 1 stop, the segment will show details about the stopover such as the length of time and where the stopover location is.
* Carriers:	Similar to places, carriers contains information about the airlines referenced in itineraries.
* Agents:	similar to places, agents contains information about the OTAs referenced in itineraries.

https://developers.skyscanner.net/docs/flights-live-prices/overview

### Itineraries

In [90]:
def clean_itineraries_table(flight_tables):
    
    # subset
    itineraries_dict = flight_tables['itineraries']
    # check if no itineraries
    if not itineraries_dict:
        raise Exception("No itineraries!")
    

    # format
    rows = []
    for key, value in itineraries_dict.items():
        for pricing_option in value['pricingOptions']:
            price = pricing_option['price']
            for item in pricing_option['items']:
                item_price = item['price']
                for fare in item['fares']:
                    rows.append({
                        'id': key,
                        'price_amount': price['amount'],
                        'price_unit': price['unit'],
                        'price_update_status': price['updateStatus'],
                        'agent_id': item['agentId'],
                        'deep_link': item['deepLink'],
                        'segment_id': fare['segmentId'],
                        'booking_code': fare['bookingCode'],
                        'fare_basis_code': fare['fareBasisCode'],
                        'transfer_type': pricing_option['transferType'],
                        'option_id': pricing_option['id']
                    })

    # create dataframe
    return pd.DataFrame(rows)


# test
itineraries = clean_itineraries_table(flight_tables)
display(itineraries.head())


,id,price_amount,price_unit,price_update_status,agent_id,deep_link,segment_id,booking_code,fare_basis_code,transfer_type,option_id
0,"14788-2305220920--32128,-32301-2-16995-2305222330",1003560,PRICE_UNIT_MILLI,PRICE_UPDATE_STATUS_UNSPECIFIED,edus,https://skyscanner.pxf.io/c/2850210/1103265/13...,14788-12409-2305220920-2305221145--32128,,,TRANSFER_TYPE_PROTECTED_SELF_TRANSFER,BNvJtlhvqMlN
1,"14788-2305220920--32128,-32301-2-16995-2305222330",1003560,PRICE_UNIT_MILLI,PRICE_UPDATE_STATUS_UNSPECIFIED,edus,https://skyscanner.pxf.io/c/2850210/1103265/13...,12409-11182-2305221330-2305221855--32128,,,TRANSFER_TYPE_PROTECTED_SELF_TRANSFER,BNvJtlhvqMlN
2,"14788-2305220920--32128,-32301-2-16995-2305222330",1003560,PRICE_UNIT_MILLI,PRICE_UPDATE_STATUS_UNSPECIFIED,edus,https://skyscanner.pxf.io/c/2850210/1103265/13...,11182-16995-2305222055-2305222330--32301,,,TRANSFER_TYPE_PROTECTED_SELF_TRANSFER,BNvJtlhvqMlN
3,14788-2305221035--31734-1-16995-2305222155,1039990,PRICE_UNIT_MILLI,PRICE_UPDATE_STATUS_UNSPECIFIED,fnus,https://skyscanner.pxf.io/c/2850210/1103265/13...,14788-12585-2305221035-2305221750--31734,,,TRANSFER_TYPE_MANAGED,VoKv1Wadljp0
4,14788-2305221035--31734-1-16995-2305222155,1039990,PRICE_UNIT_MILLI,PRICE_UPDATE_STATUS_UNSPECIFIED,fnus,https://skyscanner.pxf.io/c/2850210/1103265/13...,12585-16995-2305221950-2305222155--31734,,,TRANSFER_TYPE_MANAGED,VoKv1Wadljp0


### Legs

In [120]:
def clean_legs_table(flight_tables):
    # subset
    legs_dict = flight_tables['legs']

    # format
    rows = []
    for key, value in legs_dict.items():
        departure_dt = value['departureDateTime']
        arrival_dt = value['arrivalDateTime']
        for segment_id, marketing_carrier_id, operating_carrier_id in zip(value['segmentIds'], value['marketingCarrierIds'], value['operatingCarrierIds']):
            rows.append({
                'id': key,
                'origin_place_id': value['originPlaceId'],
                'destination_place_id': value['destinationPlaceId'],
                'departure_year': departure_dt['year'],
                'departure_month': departure_dt['month'],
                'departure_day': departure_dt['day'],
                'departure_hour': departure_dt['hour'],
                'departure_minute': departure_dt['minute'],
                'departure_second': departure_dt['second'],
                'arrival_year': arrival_dt['year'],
                'arrival_month': arrival_dt['month'],
                'arrival_day': arrival_dt['day'],
                'arrival_hour': arrival_dt['hour'],
                'arrival_minute': arrival_dt['minute'],
                'arrival_second': arrival_dt['second'],
                'duration_minutes': value['durationInMinutes'],
                'stop_count': value['stopCount'],
                'marketing_carrier_id': marketing_carrier_id,
                'operating_carrier_id': operating_carrier_id,
                'segment_id': segment_id
            })

    # create dataframe
    legs = pd.DataFrame(rows)

    # combine time  columns
    combine_to_datetime(legs, 'departure')
    combine_to_datetime(legs, 'arrival')

    # rename id column
    return legs.rename(columns={'id': 'leg_id'})


# test
legs = clean_legs_table(flight_tables)
display(legs.head(100))


,leg_id,origin_place_id,destination_place_id,duration_minutes,stop_count,marketing_carrier_id,operating_carrier_id,segment_id,departure_datetime,arrival_datetime
0,"14788-2305220915--31461,-32213,-31669-3-16995-...",128668889,95673635,1895,3,-31461,-31461,14788-9970-2305220915-2305221400--31461,2023-05-22 09:15:00,2023-05-23 10:50:00
1,"14788-2305220915--31461,-32213,-31669-3-16995-...",128668889,95673635,1895,3,-32213,-32213,9970-10075-2305221550-2305221835--32213,2023-05-22 09:15:00,2023-05-23 10:50:00
2,"14788-2305220915--31461,-32213,-31669-3-16995-...",128668889,95673635,1895,3,-31669,-32761,10075-9618-2305222215-2305222345--32213,2023-05-22 09:15:00,2023-05-23 10:50:00
3,"14788-2305220930--31705,-32339,-31669-3-16995-...",128668889,95673635,1880,3,-31705,-31705,14788-12071-2305220930-2305221305--31705,2023-05-22 09:30:00,2023-05-23 10:50:00
4,"14788-2305220930--31705,-32339,-31669-3-16995-...",128668889,95673635,1880,3,-32339,-32339,12071-10075-2305221820-2305222150--31705,2023-05-22 09:30:00,2023-05-23 10:50:00
5,"14788-2305220930--31705,-32339,-31669-3-16995-...",128668889,95673635,1880,3,-31669,-32761,10075-9618-2305230445-2305230610--32339,2023-05-22 09:30:00,2023-05-23 10:50:00
6,14788-2305221035--31734-1-16995-2305222155,128668889,95673635,1040,1,-31734,-31734,14788-12585-2305221035-2305221750--31734,2023-05-22 10:35:00,2023-05-22 21:55:00
7,"14788-2305221445--31461,-32352-1-16995-2305230445",128668889,95673635,1200,1,-31461,-31461,14788-9970-2305221445-2305221935--31461,2023-05-22 14:45:00,2023-05-23 04:45:00
8,"14788-2305221445--31461,-32352-1-16995-2305230445",128668889,95673635,1200,1,-32352,-32352,9970-16995-2305222355-2305230445--32352,2023-05-22 14:45:00,2023-05-23 04:45:00
9,"14788-2305221700--30816,-32348-2-16995-2305230935",128668889,95673635,1355,2,-30816,-30816,14788-9970-2305221700-2305222145--30816,2023-05-22 17:00:00,2023-05-23 09:35:00


In [121]:
legs_test = legs['leg_id'].nunique()
legs_test

10

### Segments

In [112]:
def clean_segments_table(flight_tables):
    # subset
    segments_dict = flight_tables['segments']

    # format
    rows = []
    for key, value in segments_dict.items():
        departure_dt = value['departureDateTime']
        arrival_dt = value['arrivalDateTime']

        rows.append({
            'id': key,
            'origin_place_id': value['originPlaceId'],
            'destination_place_id': value['destinationPlaceId'],
            'departure_year': departure_dt['year'],
            'departure_month': departure_dt['month'],
            'departure_day': departure_dt['day'],
            'departure_hour': departure_dt['hour'],
            'departure_minute': departure_dt['minute'],
            'departure_second': departure_dt['second'],
            'arrival_year': arrival_dt['year'],
            'arrival_month': arrival_dt['month'],
            'arrival_day': arrival_dt['day'],
            'arrival_hour': arrival_dt['hour'],
            'arrival_minute': arrival_dt['minute'],
            'arrival_second': arrival_dt['second'],
            'duration_minutes': value['durationInMinutes'],
            'marketing_flight_number': value['marketingFlightNumber'],
            'marketing_carrier_id': value['marketingCarrierId'],
            'operating_carrier_id': value['operatingCarrierId']
        })


    # create dataframe
    segments = pd.DataFrame(rows)

    # combine time columns
    combine_to_datetime(segments, 'departure')
    combine_to_datetime(segments, 'arrival')

    # rename id column
    return segments.rename(columns={'id': 'segment_id'})


# test
segments = clean_segments_table(flight_tables)
display(segments.head(100))


,segment_id,origin_place_id,destination_place_id,duration_minutes,marketing_flight_number,marketing_carrier_id,operating_carrier_id,departure_datetime,arrival_datetime
0,11182-16995-2305222055-2305222330--32301,95673506,95673635,215,1807,-32301,-32301,2023-05-22 20:55:00,2023-05-22 23:30:00
1,11182-16995-2305230705-2305230935--32348,95673506,95673635,210,2369,-32348,-32301,2023-05-23 07:05:00,2023-05-23 09:35:00
2,12191-16995-2305230220-2305230745--32456,128668132,95673635,625,675,-32456,-32456,2023-05-23 02:20:00,2023-05-23 07:45:00
3,12409-11182-2305221330-2305221855--32128,95673659,95673506,625,951,-32128,-32128,2023-05-22 13:30:00,2023-05-22 18:55:00
4,12409-9252-2305230105-2305230705--32340,95673659,128668203,720,673,-32340,-32340,2023-05-23 01:05:00,2023-05-23 07:05:00
5,12585-16995-2305221950-2305222155--31734,95673323,95673635,125,864,-31734,-31734,2023-05-22 19:50:00,2023-05-22 21:55:00
6,14788-11182-2305222230-2305230445--32348,128668889,95673506,675,319,-32348,-32348,2023-05-22 22:30:00,2023-05-23 04:45:00
7,14788-12191-2305221715-2305222120--32456,128668889,128668132,305,521,-32456,-32456,2023-05-22 17:15:00,2023-05-22 21:20:00
8,14788-12191-2305221830-2305222225--32456,128668889,128668132,295,505,-32456,-32456,2023-05-22 18:30:00,2023-05-22 22:25:00
9,14788-12409-2305220920-2305221145--32128,128668889,95673659,145,706,-32128,-32128,2023-05-22 09:20:00,2023-05-22 11:45:00


In [111]:
segment_test = segments['segment_id'].nunique()
segment_test

19

### Places

In [94]:
def clean_places_table(flight_tables):
    # subset
    places_dict = flight_tables['places']

    # format
    rows = []
    for key, value in places_dict.items():
        rows.append({
            'entity_id': value['entityId'],
            'parent_id': value['parentId'],
            'name': value['name'],
            'place_type': value['type'],
            'iata': value['iata'],
            'coordinates': value['coordinates']
        })


    # create dataframe
    return pd.DataFrame(rows)


# test
places = clean_places_table(flight_tables)
display(places.head())


,entity_id,parent_id,name,place_type,iata,coordinates
0,128668132,27542065,Hong Kong Intl,PLACE_TYPE_AIRPORT,HKG,None
1,128668203,27536445,Addis Ababa,PLACE_TYPE_AIRPORT,ADD,None
2,128668889,27542089,Tokyo Narita,PLACE_TYPE_AIRPORT,NRT,None
3,27536445,29475224,Addis Ababa,PLACE_TYPE_CITY,ADD,None
4,27536671,29475320,Bangkok,PLACE_TYPE_CITY,BKK,None


### Carriers

In [95]:
def clean_carriers_table(flight_tables):
    # subset
    carriers_dict = flight_tables['carriers']

    # format
    rows = []
    for key, value in carriers_dict.items():
        rows.append({
            'carrier_id': key,
            'name': value['name'],
            'alliance_id': value['allianceId'],
            'image_url': value['imageUrl'],
            'iata': value['iata']
        })



    # create dataframe
    return pd.DataFrame(rows)


# test
carriers = clean_carriers_table(flight_tables)
display(carriers.head())

,carrier_id,name,alliance_id,image_url,iata
0,-30816,Zipair,,https://logos.skyscnr.com/images/airlines/_Z.png,ZG
1,-31461,Thai AirAsia X,,https://logos.skyscnr.com/images/airlines/XD.png,XJ
2,-31734,Turkish Airlines,-31999,https://logos.skyscnr.com/images/airlines/TK.png,TK
3,-32093,LOT,-31999,https://logos.skyscnr.com/images/airlines/LO.png,LO
4,-32128,Korean Air,-31998,https://logos.skyscnr.com/images/airlines/KE.png,KE


### Agents

In [96]:
def clean_agents_table(flight_tables):
    # subset
    agents_dict = flight_tables['agents']

    # format
    rows = []
    for key, value in agents_dict.items():
        rating_breakdown = value.get('ratingBreakdown') or {}
        rows.append({
            'agent_id': key,
            'name': value['name'],
            'agent_type': value['type'],
            'image_url': value['imageUrl'],
            'feedback_count': value['feedbackCount'],
            'rating': value['rating'],
            'customer_service': rating_breakdown.get('customerService', None),
            'reliable_prices': rating_breakdown.get('reliablePrices', None),
            'clear_extra_fees': rating_breakdown.get('clearExtraFees', None),
            'ease_of_booking': rating_breakdown.get('easeOfBooking', None),
            'other': rating_breakdown.get('other', None),
            'is_optimised_for_mobile': value['isOptimisedForMobile']
        })

    # create dataframe
    return pd.DataFrame(rows)


# test
agents = clean_agents_table(flight_tables)
display(agents.head())


,agent_id,name,agent_type,image_url,feedback_count,rating,customer_service,reliable_prices,clear_extra_fees,ease_of_booking,other,is_optimised_for_mobile
0,airf,Air France,AGENT_TYPE_AIRLINE,https://logos.skyscnr.com/images/websites/airf...,1903,2.87,5.000000,1.668960,4.141784,2.992648,2.687180,True
1,arus,Mytrip,AGENT_TYPE_TRAVEL_AGENT,https://logos.skyscnr.com/images/websites/arus...,8424,3.25,4.856892,3.538052,4.064076,3.325836,2.217688,True
2,ausa,Austrian Airlines,AGENT_TYPE_AIRLINE,https://logos.skyscnr.com/images/websites/ausa...,364,3.75,5.000000,3.303972,4.537448,3.612340,3.535248,True
3,bcom,Booking.com,AGENT_TYPE_TRAVEL_AGENT,https://logos.skyscnr.com/images/websites/bcom...,4381,3.62,4.983912,3.188784,4.415532,3.849500,3.050716,True
4,bfus,Bravofly,AGENT_TYPE_TRAVEL_AGENT,https://logos.skyscnr.com/images/websites/bfus...,2773,2.62,4.711384,2.962752,3.976468,1.484060,2.339328,True


### Alliances

In [97]:
def clean_alliances_table(flight_tables):
    # subset
    alliances_dict = flight_tables['alliances']

    # format
    rows = []
    for key, value in alliances_dict.items():
        rows.append({
            'alliance_id': key,
            'name': value['name']
        })

    # create dataframe
    return pd.DataFrame(rows)


# test
alliances = clean_alliances_table(flight_tables)
display(alliances.head())



,alliance_id,name
0,-31998,SkyTeam
1,-31999,Star Alliance
2,-32000,OneWorld


## Combine

In [107]:
def create_full_journey_table(departure_airport, arrival_airport, itineraries_table, agents_table, segments_table, carriers_table, places_table):
    
    trips_df = itineraries_table
    
    # sort by id
    #trips_df = itineraries_table.sort_values(by="id", ascending=True)
    #trips_df = trips_df.sort_values(by="price_amount", ascending=True)
    
    # temp column to preserve order
    # Add a temporary column to store the original index
    trips_df['temp_index'] = trips_df.index
    
    
    # price to float
    trips_df['price_amount'] = trips_df['price_amount'].astype(float)

    # drop unecesary columns
    trips_df = trips_df.drop(['deep_link', 'option_id', 'booking_code', 'fare_basis_code', 'price_update_status'], axis=1)


    # bring in the agent name
    trips_df = pd.merge(trips_df, agents_table[['agent_id', 'name']], on='agent_id')
    trips_df = trips_df.rename(columns={'name': 'agent_name'})
    trips_df = trips_df.drop('agent_id', axis=1)

    # bring in segment info
    trips_df = pd.merge(trips_df, segments_table, on='segment_id')

    # get carrier names
    trips_df = pd.merge(trips_df, carriers_table[['carrier_id', 'name']], left_on='marketing_carrier_id', right_on='carrier_id')
    trips_df = trips_df.rename(columns={'name': 'marketing_carrier_name'})
    trips_df = pd.merge(trips_df, carriers_table[['carrier_id', 'name']], left_on='operating_carrier_id', right_on='carrier_id')
    trips_df = trips_df.rename(columns={'name': 'operating_carrier_name'})
    trips_df = trips_df.drop(['marketing_carrier_id', 'operating_carrier_id', 'carrier_id_x', 'carrier_id_y'], axis=1)

    # bring in places
    trips_df = pd.merge(trips_df, places_table[['entity_id', 'name', 'iata']], left_on='origin_place_id', right_on='entity_id')
    trips_df = trips_df.rename(columns={'name': 'origin_place_name', 'iata': 'origin_place_iata' })
    trips_df = pd.merge(trips_df, places_table[['entity_id', 'name', 'iata']], left_on='destination_place_id', right_on='entity_id')
    trips_df = trips_df.rename(columns={'name': 'destination_place_name', 'iata': 'destination_place_iata'})
    trips_df = trips_df.drop(['origin_place_id', 'destination_place_id', 'entity_id_x', 'entity_id_y'], axis=1)

    # results don't only show airports requested, remove unrequested airports
    #trips_df = trips_df[trips_df['origin_place_iata'] == departure_airport]
    #trips_df = trips_df[trips_df['destination_place_iata'] == arrival_airport]
    
    # sort and drop temp column
    # Sort the merged DataFrame based on the temporary column
    trips_df = trips_df.sort_values('temp_index')

    # Drop the temporary column
    trips_df = trips_df.drop('temp_index', axis=1)
    
    #trips_df = trips_df.sort_values(by=['price_amount', 'id'], ascending=[True, True])

    return trips_df


# test
trips_df = create_full_journey_table('NRT', 'TPE', itineraries, agents, segments, carriers, places)
display(trips_df.head(100))


,id,price_amount,price_unit,segment_id,transfer_type,agent_name,duration_minutes,marketing_flight_number,departure_datetime,arrival_datetime,marketing_carrier_name,operating_carrier_name,origin_place_name,origin_place_iata,destination_place_name,destination_place_iata
0,"14788-2305220920--32128,-32301-2-16995-2305222330",1003560.0,PRICE_UNIT_MILLI,14788-12409-2305220920-2305221145--32128,TRANSFER_TYPE_PROTECTED_SELF_TRANSFER,eDreams,145,706,2023-05-22 09:20:00,2023-05-22 11:45:00,Korean Air,Korean Air,Tokyo Narita,NRT,Incheon International Airport,ICN
7,"14788-2305220920--32128,-32301-2-16995-2305222330",1003560.0,PRICE_UNIT_MILLI,12409-11182-2305221330-2305221855--32128,TRANSFER_TYPE_PROTECTED_SELF_TRANSFER,eDreams,625,951,2023-05-22 13:30:00,2023-05-22 18:55:00,Korean Air,Korean Air,Incheon International Airport,ICN,Dubai,DXB
13,"14788-2305220920--32128,-32301-2-16995-2305222330",1003560.0,PRICE_UNIT_MILLI,11182-16995-2305222055-2305222330--32301,TRANSFER_TYPE_PROTECTED_SELF_TRANSFER,eDreams,215,1807,2023-05-22 20:55:00,2023-05-22 23:30:00,flydubai,flydubai,Dubai,DXB,Ben Gurion Intl,TLV
2,14788-2305221035--31734-1-16995-2305222155,1039990.0,PRICE_UNIT_MILLI,14788-12585-2305221035-2305221750--31734,TRANSFER_TYPE_MANAGED,FlightNetwork,795,51,2023-05-22 10:35:00,2023-05-22 17:50:00,Turkish Airlines,Turkish Airlines,Tokyo Narita,NRT,Istanbul,IST
16,14788-2305221035--31734-1-16995-2305222155,1039990.0,PRICE_UNIT_MILLI,12585-16995-2305221950-2305222155--31734,TRANSFER_TYPE_MANAGED,FlightNetwork,125,864,2023-05-22 19:50:00,2023-05-22 21:55:00,Turkish Airlines,Turkish Airlines,Istanbul,IST,Ben Gurion Intl,TLV
3,"14788-2305221445--31461,-32352-1-16995-2305230445",919990.0,PRICE_UNIT_MILLI,14788-9970-2305221445-2305221935--31461,TRANSFER_TYPE_PROTECTED_SELF_TRANSFER,Mytrip,410,603,2023-05-22 14:45:00,2023-05-22 19:35:00,Thai AirAsia X,Thai AirAsia X,Tokyo Narita,NRT,Bangkok Suvarnabhumi,BKK
17,"14788-2305221445--31461,-32352-1-16995-2305230445",919990.0,PRICE_UNIT_MILLI,9970-16995-2305222355-2305230445--32352,TRANSFER_TYPE_PROTECTED_SELF_TRANSFER,Mytrip,530,82,2023-05-22 23:55:00,2023-05-23 04:45:00,EL AL Israel Airlines,EL AL Israel Airlines,Bangkok Suvarnabhumi,BKK,Ben Gurion Intl,TLV
4,"14788-2305221700--30816,-32348-2-16995-2305230935",855600.0,PRICE_UNIT_MILLI,14788-9970-2305221700-2305222145--30816,TRANSFER_TYPE_PROTECTED_SELF_TRANSFER,Trip.com,405,51,2023-05-22 17:00:00,2023-05-22 21:45:00,Zipair,Zipair,Tokyo Narita,NRT,Bangkok Suvarnabhumi,BKK
8,"14788-2305221700--30816,-32348-2-16995-2305230935",855600.0,PRICE_UNIT_MILLI,9970-11182-2305230115-2305230445--32348,TRANSFER_TYPE_PROTECTED_SELF_TRANSFER,Trip.com,390,385,2023-05-23 01:15:00,2023-05-23 04:45:00,Emirates,Emirates,Bangkok Suvarnabhumi,BKK,Dubai,DXB
14,"14788-2305221700--30816,-32348-2-16995-2305230935",855600.0,PRICE_UNIT_MILLI,11182-16995-2305230705-2305230935--32348,TRANSFER_TYPE_PROTECTED_SELF_TRANSFER,Trip.com,210,2369,2023-05-23 07:05:00,2023-05-23 09:35:00,Emirates,flydubai,Dubai,DXB,Ben Gurion Intl,TLV


## Single function

In [124]:
# set departure and arrival airports
departure_airport = 'NRT'
arrival_airport = 'TLV'

def get_flight_data(departure_airport, arrival_airport, year, month, day):

    flight_tables = get_flight_tables(departure_airport, arrival_airport, year, month, day)

    itineraries_table = clean_itineraries_table(flight_tables)
    agents_table = clean_agents_table(flight_tables)
    segments_table = clean_segments_table(flight_tables)
    carriers_table = clean_carriers_table(flight_tables)
    places_table =clean_places_table(flight_tables)

    return create_full_journey_table(departure_airport, arrival_airport, itineraries_table, agents_table, segments_table, carriers_table, places_table)

get_flight_data(departure_airport, arrival_airport, 2023, 5, 22)

,id,price_amount,price_unit,segment_id,transfer_type,agent_name,duration_minutes,marketing_flight_number,departure_datetime,arrival_datetime,marketing_carrier_name,operating_carrier_name,origin_place_name,origin_place_iata,destination_place_name,destination_place_iata
0,"14788-2305220915--31461,-32213,-31669-3-16995-...",619990.0,PRICE_UNIT_MILLI,14788-9970-2305220915-2305221400--31461,TRANSFER_TYPE_PROTECTED_SELF_TRANSFER,Mytrip,405,601,2023-05-22 09:15:00,2023-05-22 14:00:00,Thai AirAsia X,Thai AirAsia X,Tokyo Narita,NRT,Bangkok Suvarnabhumi,BKK
11,"14788-2305220915--31461,-32213,-31669-3-16995-...",619990.0,PRICE_UNIT_MILLI,9970-10075-2305221550-2305221835--32213,TRANSFER_TYPE_PROTECTED_SELF_TRANSFER,Mytrip,255,1052,2023-05-22 15:50:00,2023-05-22 18:35:00,IndiGo,IndiGo,Bangkok Suvarnabhumi,BKK,Mumbai,BOM
23,"14788-2305220915--31461,-32213,-31669-3-16995-...",619990.0,PRICE_UNIT_MILLI,10075-9618-2305222215-2305222345--32213,TRANSFER_TYPE_PROTECTED_SELF_TRANSFER,Mytrip,180,1401,2023-05-22 22:15:00,2023-05-22 23:45:00,IndiGo,IndiGo,Mumbai,BOM,Abu Dhabi International,AUH
15,"14788-2305220915--31461,-32213,-31669-3-16995-...",619990.0,PRICE_UNIT_MILLI,9618-16995-2305230810-2305231050--31669,TRANSFER_TYPE_PROTECTED_SELF_TRANSFER,Mytrip,220,7085,2023-05-23 08:10:00,2023-05-23 10:50:00,Wizz Air,Wizz Air Abu Dhabi,Abu Dhabi International,AUH,Ben Gurion Intl,TLV
4,"14788-2305220930--31705,-32339,-31669-3-16995-...",560990.0,PRICE_UNIT_MILLI,14788-12071-2305220930-2305221305--31705,TRANSFER_TYPE_PROTECTED_SELF_TRANSFER,Mytrip,335,933,2023-05-22 09:30:00,2023-05-22 13:05:00,VietJet Air,VietJet Air,Tokyo Narita,NRT,Hanoi,HAN
12,"14788-2305220930--31705,-32339,-31669-3-16995-...",560990.0,PRICE_UNIT_MILLI,12071-10075-2305221820-2305222150--31705,TRANSFER_TYPE_PROTECTED_SELF_TRANSFER,Mytrip,300,907,2023-05-22 18:20:00,2023-05-22 21:50:00,VietJet Air,VietJet Air,Hanoi,HAN,Mumbai,BOM
24,"14788-2305220930--31705,-32339,-31669-3-16995-...",560990.0,PRICE_UNIT_MILLI,10075-9618-2305230445-2305230610--32339,TRANSFER_TYPE_PROTECTED_SELF_TRANSFER,Mytrip,175,205,2023-05-23 04:45:00,2023-05-23 06:10:00,Etihad Airways,Etihad Airways,Mumbai,BOM,Abu Dhabi International,AUH
16,"14788-2305220930--31705,-32339,-31669-3-16995-...",560990.0,PRICE_UNIT_MILLI,9618-16995-2305230810-2305231050--31669,TRANSFER_TYPE_PROTECTED_SELF_TRANSFER,Mytrip,220,7085,2023-05-23 08:10:00,2023-05-23 10:50:00,Wizz Air,Wizz Air Abu Dhabi,Abu Dhabi International,AUH,Ben Gurion Intl,TLV
5,14788-2305221035--31734-1-16995-2305222155,1039990.0,PRICE_UNIT_MILLI,14788-12585-2305221035-2305221750--31734,TRANSFER_TYPE_MANAGED,FlightNetwork,795,51,2023-05-22 10:35:00,2023-05-22 17:50:00,Turkish Airlines,Turkish Airlines,Tokyo Narita,NRT,Istanbul,IST
17,14788-2305221035--31734-1-16995-2305222155,1039990.0,PRICE_UNIT_MILLI,12585-16995-2305221950-2305222155--31734,TRANSFER_TYPE_MANAGED,FlightNetwork,125,864,2023-05-22 19:50:00,2023-05-22 21:55:00,Turkish Airlines,Turkish Airlines,Istanbul,IST,Ben Gurion Intl,TLV


## Loop

In [80]:
iata_codes_list = capital_cities['airport_iata'].tolist()
#iata_codes_list[:5]
iata_codes_list

['KBL',
 'EVN',
 'GYD',
 'BAH',
 'DAC',
 'BWN',
 'PNH',
 'PEK',
 'LCA',
 'DIL',
 'TBS',
 'DEL',
 'CGK',
 'IKA',
 'BGW',
 'TLV',
 'NRT',
 'AMM',
 'NQZ',
 'KWI',
 'FRU',
 'VTE',
 'BEY',
 'KUL',
 'MLE',
 'UBN',
 'NYT',
 'KTM',
 'MCT',
 'ISB',
 'MNL',
 'DOH',
 'RUH',
 'SIN',
 'ICN',
 'CMB',
 'DAM',
 'TPE',
 'DYU',
 'BKK',
 'ESB',
 'ASB',
 'AUH',
 'TAS',
 'HAN']

In [81]:
# get airport pairs without departing and arriving at same place
airport_pairs = [(iata_codes_list[i], iata_codes_list[j]) for i in range(len(iata_codes_list)) for j in range(len(iata_codes_list)) if i != j]
display(len(iata_codes_list))
display(len(airport_pairs))


45

1980

In [82]:
# use shorter if needed
if True == False:
    airport_pairs_for_run = airport_pairs[-5:]
    len(airport_pairs_for_run)
    display(airport_pairs_for_run)
else:
    airport_pairs_for_run = airport_pairs
    len(airport_pairs_for_run)
    display(airport_pairs_for_run[:5])



[('KBL', 'EVN'),
 ('KBL', 'GYD'),
 ('KBL', 'BAH'),
 ('KBL', 'DAC'),
 ('KBL', 'BWN')]

In [128]:
# timestamp so all files have the same time
timestamp = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
#timestamp = '2023-04-26_03-36-24'

timestamped_directory = f'../Data/FlightAPIData/{timestamp}/'
if not os.path.exists(timestamped_directory):
    os.makedirs(timestamped_directory)

time_sleep_sec = 2
year = 2023
month = 5
day = 22

for departure in iata_codes_list:



    if departure in ['KBL', 'EVN', 'GYD', 'BAH', 'DAC', 'DAC','BWN','PNH','PEK', 'LCA', 'DIL', 'TBS', 'DEL', 'CGK', 'IKA', 'BGW','TLV','NRT','AMM','NQZ','KWI','FRU','VTE','BEY','KUL','MLE']:
        continue
    
    # init empty list
    dataframes = []
    
    for arrival in iata_codes_list:
        
        # skip if departure and arrival airport are the same
        if departure == arrival:
            continue
        
        
        try:
            print(f'trying - departure: {departure}, arrival: {arrival}')
            df = get_flight_data(departure, arrival, year, month, day)
            df['departure_query'] = departure
            df['arrival_query'] = arrival
            dataframes.append(df)
            print('success')
            
            # sleep to avoid hitting the api limit
            time.sleep(time_sleep_sec)
        except Exception as e:
            print(f"error while processing {departure}-{arrival}: {e}")
            # sleep to avoid hitting the api limit
            time.sleep(time_sleep_sec)
            continue
    
    # concat the dataframes
    combined_dataframe = pd.concat(dataframes, ignore_index=True)
    #combine_to_datetime(combined_dataframe, 'arrival')
    #combine_to_datetime(combined_dataframe, 'departure')

    # save
    combined_dataframe.to_csv(f'../Data/FlightAPIData/{timestamp}/{timestamp}_{departure}_flight_api_data.csv', index=False)

# show
display(combined_dataframe.head())            

trying - departure: BGW, arrival: KBL
error while processing BGW-KBL: No itineraries!
trying - departure: BGW, arrival: EVN
success
trying - departure: BGW, arrival: GYD
success
trying - departure: BGW, arrival: BAH
success
trying - departure: BGW, arrival: DAC
success
trying - departure: BGW, arrival: BWN
success
trying - departure: BGW, arrival: PNH
success
trying - departure: BGW, arrival: PEK
success
trying - departure: BGW, arrival: LCA
success
trying - departure: BGW, arrival: DIL
success
trying - departure: BGW, arrival: TBS
success
trying - departure: BGW, arrival: DEL
success
trying - departure: BGW, arrival: CGK
success
trying - departure: BGW, arrival: IKA
success
trying - departure: BGW, arrival: TLV
success
trying - departure: BGW, arrival: NRT
success
trying - departure: BGW, arrival: AMM
success
trying - departure: BGW, arrival: NQZ
success
trying - departure: BGW, arrival: KWI
success
trying - departure: BGW, arrival: FRU
success
trying - departure: BGW, arrival: VTE
su

ValueError: No objects to concatenate